Import all the libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

In [77]:
train_data = pd.read_csv('Warm_Up_Predict_Blood_Donations_-_Traning_Data.csv',
                         index_col=0, dtype = "float64")
test_data = pd.read_csv('Warm_Up_Predict_Blood_Donations_-_Test_Data.csv',
                        index_col=0)
submission_format = pd.read_csv('Warm_Up_Predict_Blood_Donations_-_Submission_Format.csv',
                        index_col=0)

In [78]:
y_train = train_data["Made Donation in March 2007"]
train_data = train_data.drop("Made Donation in March 2007", axis=1)

In [79]:
##describe the data, each columns and why we ahve to drop volume

In [80]:
def fea_gen(data):
    data["donation frequency"] = (data["Months since First Donation"] - data["Months since Last Donation"])/data["Number of Donations"]
    data["last / average"] = data["Months since Last Donation"] / data["donation frequency"]
    data = data.drop("Total Volume Donated (c.c.)", axis=1)   
    data["last / average"] = data["last / average"].replace(np.inf, 5)
    data.reset_index(drop=True, inplace=True)
    poly = PolynomialFeatures(degree=3, interaction_only=True)
    poly_fea = pd.DataFrame(poly.fit_transform(data))
    poly_fea.reset_index(drop=True, inplace=True)
    data = pd.concat([data, poly_fea], axis=1)
    del poly_fea
    return data

In [81]:
x_train = fea_gen(train_data)

In [82]:
test_data = fea_gen(test_data)

In [66]:
#this is going to be a benchmark, explain why

In [67]:
len(y_train[y_train == 0])/float(len(y_train))

0.7604166666666666

In [68]:
StatifiedCV = StratifiedKFold(y            = y_train, 
                              n_folds      = 5, 
                              shuffle      = True, 
                              random_state = 13)

In [69]:
est1 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est1.fit(x_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [70]:
est1.best_score_

0.78125

In [93]:
log1 = LogisticRegression(tol=1e-20).fit(x_train,y_train)

In [94]:
from sklearn.metrics import accuracy_score

In [95]:
accuracy_score(y_train,log1.predict(x_train))

0.79861111111111116

In [96]:
submission_format["Made Donation in March 2007"] = log1.predict_proba(test_data)[:,1]

In [97]:
submission_format.to_csv("log l1 then l2 - CV 78.64.csv")

In [13]:
#potentially using regularization we can get this score even higher, but we also might consider poly features
#and then using regularization generate sparse coefficients

In [14]:
#let's consider poly features and try to get a higher CV mean accuracy score


In [15]:
def fea_gen_int(data):
    data["donation frequency"] = (data["Months since First Donation"] - data["Months since Last Donation"])/data["Number of Donations"]
    data["last / average"] = data["Months since Last Donation"] / data["donation frequency"]
    data = data.drop("Total Volume Donated (c.c.)", axis=1)   
    data["last / average"] = data["last / average"].replace(np.inf, 5)
    data.reset_index(drop=True, inplace=True)
    poly = PolynomialFeatures(degree=4, interaction_only=False)
    poly_fea = pd.DataFrame(poly.fit_transform(data))
    poly_fea.reset_index(drop=True, inplace=True)
    data = pd.concat([data, poly_fea], axis=1)
    del poly_fea
    return data

In [16]:
x_train_int = fea_gen_int(train_data)

In [17]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.1s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [18]:
est2.best_score_

0.76388888888888884

In [19]:
#the score is lower, but we might potentially get it higher using L1 and L2 regularization

In [20]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={"C":np.linspace(0.1,1,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.1min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 0.325, 0.55, 0.775, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [21]:
est2.best_score_

0.765625

In [22]:
est2.grid_scores_

[mean: 0.75347, std: 0.02047, params: {'C': 0.1},
 mean: 0.76562, std: 0.03006, params: {'C': 0.325},
 mean: 0.76042, std: 0.02741, params: {'C': 0.55},
 mean: 0.76042, std: 0.02741, params: {'C': 0.775},
 mean: 0.75521, std: 0.02809, params: {'C': 1.0}]

In [23]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={"C":np.linspace(1e-4,0.325,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   48.9s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.08132500000000001, 0.16255, 0.24377500000000002, 0.325]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [24]:
est2.grid_scores_

[mean: 0.77778, std: 0.03339, params: {'C': 0.0001},
 mean: 0.76736, std: 0.02915, params: {'C': 0.08132500000000001},
 mean: 0.76389, std: 0.02570, params: {'C': 0.16255},
 mean: 0.76215, std: 0.02937, params: {'C': 0.24377500000000002},
 mean: 0.76910, std: 0.02928, params: {'C': 0.325}]

In [25]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={"C":np.linspace(1e-10,0.1,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   39.4s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1e-10, 0.025000000075000004, 0.05000000005000001, 0.075000000025, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [26]:
est2.grid_scores_

[mean: 0.76042, std: 0.00222, params: {'C': 1e-10},
 mean: 0.77604, std: 0.02672, params: {'C': 0.025000000075000004},
 mean: 0.76910, std: 0.02421, params: {'C': 0.05000000005000001},
 mean: 0.76562, std: 0.02557, params: {'C': 0.075000000025},
 mean: 0.76389, std: 0.02223, params: {'C': 0.1}]

In [27]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={"C":np.linspace(1e-9,0.1,10).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.6min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1e-09, 0.011111112, 0.022222223, 0.033333334, 0.044444445, 0.055555556, 0.066666667, 0.07777777799999999, 0.088888889, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [28]:
est2.grid_scores_

[mean: 0.76042, std: 0.00222, params: {'C': 1e-09},
 mean: 0.76389, std: 0.03112, params: {'C': 0.011111112},
 mean: 0.76736, std: 0.02399, params: {'C': 0.022222223},
 mean: 0.77083, std: 0.02920, params: {'C': 0.033333334},
 mean: 0.76910, std: 0.01307, params: {'C': 0.044444445},
 mean: 0.76736, std: 0.02983, params: {'C': 0.055555556},
 mean: 0.76215, std: 0.02375, params: {'C': 0.066666667},
 mean: 0.76562, std: 0.02528, params: {'C': 0.07777777799999999},
 mean: 0.76215, std: 0.02157, params: {'C': 0.088888889},
 mean: 0.76215, std: 0.02157, params: {'C': 0.1}]

In [29]:
est2 = GridSearchCV(estimator=LogisticRegression(penalty="l1"),
                    param_grid={"C":[1e-4,1e-6,1e-5]},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est2.fit(x_train_int,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.8s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 1e-06, 1e-05]}, pre_dispatch='2*n_jobs',
       refit=True, scoring=None, verbose=1)

In [30]:
est2.grid_scores_

[mean: 0.77951, std: 0.03069, params: {'C': 0.0001},
 mean: 0.76736, std: 0.00612, params: {'C': 1e-06},
 mean: 0.73438, std: 0.02829, params: {'C': 1e-05}]

In [31]:
est3 = GridSearchCV(estimator=LogisticRegression(penalty="l2"),
                    param_grid={"C":np.linspace(0.3,0.4,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est3.fit(x_train_int,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    5.4s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.3, 0.325, 0.35, 0.375, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [32]:
est3.grid_scores_

[mean: 0.77951, std: 0.03134, params: {'C': 0.3},
 mean: 0.77778, std: 0.02929, params: {'C': 0.325},
 mean: 0.78125, std: 0.03571, params: {'C': 0.35},
 mean: 0.78299, std: 0.03112, params: {'C': 0.375},
 mean: 0.77778, std: 0.03229, params: {'C': 0.4}]

In [33]:
simple_log =LogisticRegression(penalty="l1", C=0.5).fit(x_train_int,y_train)

In [34]:
temp_x = x_train_int.ix[:,simple_log.coef_[0]!=0]

In [35]:
est4 = GridSearchCV(estimator=LogisticRegression(penalty="l2"),
                    param_grid={"C":np.linspace(0.1,1,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=1)
est4.fit(temp_x,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    4.8s finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  0.  0.], n_folds=5, shuffle=True, random_state=13),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 0.325, 0.55, 0.775, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [36]:
print est4.best_score_, est4.best_params_

0.779513888889 {'C': 0.325}


In [37]:
for i in np.linspace(0.3,0.7,5).tolist():
    simple_log =LogisticRegression(penalty="l1", C=i).fit(x_train_int,y_train)
    temp_x = x_train_int.ix[:,simple_log.coef_[0]!=0]
    est4 = GridSearchCV(estimator=LogisticRegression(penalty="l2"),
                    param_grid={"C":np.linspace(0.1,1,5).tolist()},
                    n_jobs=1,
                    cv=StatifiedCV,
                    verbose=0)
    est4.fit(temp_x,y_train)
    print i ,"and",est4.best_score_, "and",est4.best_params_

0.3 and 0.779513888889 and {'C': 0.1}
0.4 and 0.786458333333 and {'C': 0.325}
0.5 and 0.782986111111 and {'C': 0.775}
0.6 and 0.779513888889 and {'C': 0.55}
0.7 and 0.782986111111 and {'C': 0.775}


In [38]:
log_fin = LogisticRegression(penalty="l1", C=0.6).fit(x_train_int,y_train)

In [39]:
x_fin = x_train_int.ix[:,log_fin.coef_[0]!=0]

In [40]:
log = LogisticRegression(penalty="l2", C=0.325).fit(x_fin,y_train)

In [75]:
test_data = fea_gen_int(test_data)

In [42]:
test_data = test_data.ix[:,log_fin.coef_[0]!=0]

In [43]:
submission_format["Made Donation in March 2007"] = log.predict_proba(test_data)[:,1]

In [44]:
submission_format.to_csv("log l1 then l2 - CV 78.64.csv")

In [45]:
0.5182

0.5182

In [46]:
log1 = LogisticRegression().fit(x_train,y_train)

In [47]:
test_data = fea_gen(test_data)

KeyError: 'Months since Last Donation'

In [ ]:
submission_format["Made Donation in March 2007"] = log1.predict_proba(test_data)[:,1]

In [ ]:
submission_format.to_csv("only interactions poly 3 - 78.47.csv")

In [ ]:
0.4494